## ODS video editor
Video editor jupyter notebook

Feature `Concat audio and video parts`
- cut (start, end) part of audio and video files
- merge video and audio file with the same length
- concat video files into common file

Feature `Concat video and images frames`
- cut (start, end) part of video files
- create video sample from static image with fixed duration
- concat ordered samples to common video

Structure
- `notebooks/ods_video_converter.ipynb`
- `data`
- - `init` *.mp4 files with audio
- - `input`
- - - `audio` *.mp3 files
- - - `img` *.jpeg files
- - - `video`  *.mp4 files without audio
- - `output` *.mp4 result files with video and audio


In [2]:
import os
import moviepy.editor as mp

In [3]:
! pwd

/Users/o/PycharmProjects/ods_video_editor/notebooks


In [4]:
! ls ../data/init_hb

ww music.mp4
ww trailer 2 season.mp4
хороший плохой злой флейта.mp4
хороший плохой злой.mp4


## Set input parameters

In [6]:
input_video_dir = '../data/input/video'
input_audio_dir = '../data/input/audio'

ofile_path = '../data/output/jd_videos_all.mp4'
# set name and time slice in seconds (start_sec, end_sec)
concat_dict = {'jd_sample_yota_4': (0, 10),
               'jd_sample_yota_3': (1, 6),
              }

## Concat video and audio files

In [8]:
%%time
clips = []
for name, (start_sec, end_sec) in concat_dict.items():
    print(name)
    ifile_video_path = f'{input_video_dir}/{name}.mp4'
    ifile_audio_path = f'{input_audio_dir}/{name}.mp3'
    clip = mp.VideoFileClip(ifile_video_path).subclip(start_sec, end_sec)
    audio = mp.AudioFileClip(ifile_audio_path).subclip(start_sec, end_sec)
    clip = clip.set_audio(audio)
    clips.append(clip)

# save result video
final_clip = mp.concatenate_videoclips(clips, method='compose')
final_clip.write_videofile(ofile_path,
                           codec='libx264', 
                           audio_codec='aac', 
                           temp_audiofile='temp-audio.m4a', 
                           remove_temp=True)

jd_sample_yota_4
jd_sample_yota_3


chunk:  20%|█▉        | 65/331 [00:00<00:00, 648.76it/s, now=None]

Moviepy - Building video ../data/output/jd_videos_all.mp4.
MoviePy - Writing audio in temp-audio.m4a


t:   1%|          | 4/450 [00:00<00:13, 34.00it/s, now=None]       

MoviePy - Done.
Moviepy - Writing video ../data/output/jd_videos_all.mp4



Moviepy - Done !
Moviepy - video ready ../data/output/jd_videos_all.mp4
CPU times: user 2.35 s, sys: 2.05 s, total: 4.4 s
Wall time: 1min 14s


In [10]:
input_video_dir = '../data/init_hb'
# input_audio_dir = '../data/input/audio'

ofile_path = '../data/output/eug_hb.mp4'
# set name and time slice in seconds (start_sec, end_sec)
concat_dict = {
         'gbu_video': [
                   ('хороший плохой злой.mp4',      (230, 252), 'mp4'),
#                    ('хороший плохой злой флейта.mp4', (13, 37), 'mp3'),
                   ('хороший плохой злой dasha cover.mp4', (0, 22), 'mp3')
          ],
         'bb_video': [
              ('breakin bad say my name.mp4', (33, 43), 'mp4'),
              ('breaking bad cover.mp4', (3, 13), 'mp3')
          ],
         'ww_video':[
                ('ww trailer 2 season.mp4', (43, 50), 'mp4'),
                ('ww music.mp4', (75, 75 + 7), 'mp3'),
          ],
          'wh_video': [
              ('wh trailer.mp4', (158, 173), 'mp4'),
              ('witcher wild hunt.mp4', (15, 30), 'mp3')
          ],
          'hb main': [
              ('hb dasha main 640.mp4', (3, 21), 'all'),
              ('hb eug.png', (3, 21), 'png')
          ]
       }

# breaking bad 33 - 43
# breakin bad cover 3 - inf

# withcher cover 15 sec 
# warhammer 2:38 - 2:53

In [12]:
3 * 60 + 50, 4 * 60 + 14, 13, 13 + 24

(230, 254, 13, 37)

In [12]:
%%time
clips = []

for name, media_pair in concat_dict.items():
    print(name)
    for media_name, (start_sec, end_sec), media_type in media_pair:
        print('\t', media_name)
        if media_type == 'mp4':
            clip = mp.VideoFileClip(f'{input_video_dir}/{media_name}', audio=False).subclip(start_sec, end_sec)
        elif media_type == 'mp3':
            audio = mp.AudioFileClip(f'{input_video_dir}/{media_name}').subclip(start_sec, end_sec)
            clip = clip.set_audio(audio)
            clips.append(clip)
        elif media_type == 'png':
            logo = (mp.ImageClip(f'{input_video_dir}/{media_name}')
                      .set_duration(end_sec - start_sec)
                      .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
                      .set_pos(("right","top")))
#              .set_duration(end_sec - start_sec)
#                       .resize(height=50)
#                       .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
#                       .set_pos(("right","top")))
            clips[-1] = mp.CompositeVideoClip([clips[-1], logo])
        elif media_type == 'all':
            clip = mp.VideoFileClip(f'{input_video_dir}/{media_name}', audio=True) \
                     .subclip(start_sec, end_sec)
            clips.append(clip)
        else:
            print('unknown type', media_type)
    
#     ifile_video_path = f'{input_video_dir}/{name}.mp4'
#     ifile_audio_path = f'{input_audio_dir}/{name}.mp3'
#     clip = mp.VideoFileClip(ifile_video_path).subclip(start_sec, end_sec)
#     audio = mp.AudioFileClip(ifile_audio_path).subclip(start_sec, end_sec)
#     clip = clip.set_audio(audio)
#     clips.append(clip)

# save result video
final_clip = mp.concatenate_videoclips(clips, method='compose')
final_clip.write_videofile(ofile_path,
                           codec='libx264', 
                           audio_codec='aac', 
                           temp_audiofile='temp-audio.m4a', 
                           remove_temp=True)

gbu_video
	 хороший плохой злой.mp4
	 хороший плохой злой dasha cover.mp4
bb_video
	 breakin bad say my name.mp4
	 breaking bad cover.mp4
ww_video
	 ww trailer 2 season.mp4
	 ww music.mp4
wh_video
	 wh trailer.mp4
	 witcher wild hunt.mp4
hb main
	 hb dasha main 640.mp4


chunk:   0%|          | 0/1588 [00:00<?, ?it/s, now=None]

	 hb eug.png
Moviepy - Building video ../data/output/eug_hb.mp4.
MoviePy - Writing audio in temp-audio.m4a



t:   0%|          | 3/2160 [00:00<01:12, 29.80it/s, now=None]

MoviePy - Done.
Moviepy - Writing video ../data/output/eug_hb.mp4



...

t:  70%|███████   | 1521/2160 [00:31<00:14, 43.88it/s, now=None]


t:  85%|████████▌ | 1841/2160 [00:50<00:25, 12.29it/s, now=None]


t:  98%|█████████▊| 2115/2160 [01:08<00:04,  9.86it/s, now=None]


chunk:  29%|██▉       | 491/1698 [22:49<00:02, 518.69it/s, now=None]

Moviepy - Done !
Moviepy - video ready ../data/output/eug_hb.mp4
CPU times: user 29.6 s, sys: 5.76 s, total: 35.3 s
Wall time: 1min 18s


In [24]:
60 + 25, 60 + 25 +  17

(85, 102)

## Result video preview

In [9]:
%%time
# final_clip.ipython_display(width=280)
final_clip.resize(width=160, height=90) \
          .ipython_display(logger='none')

chunk:  10%|▉         | 33/331 [00:00<00:00, 298.22it/s, now=None]

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


t:   0%|          | 0/450 [00:00<?, ?it/s, now=None]               

MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
CPU times: user 17 s, sys: 1.59 s, total: 18.6 s
Wall time: 28.1 s


In [13]:
## Concat images and videos

In [14]:
%%time
input_video_dir = '../data/init'
input_img_dir = '../data/input/img'
ofile_video_img_path = '../data/output/jd_videos_all_img.mp4'
# set name and time slice in seconds (start_sec, end_sec)
# for images need only duration (end_sec - start_sec)
concat_img_dict = {
                   'jd_sample_yota_3': (1, 6),
                   'jd_sample_yota_2': (0, 10),
                   'img1': (0, 5),
                   'img2': (0, 5),
                   'img3': (0, 10)
                  }
clips_order = ['img1', 'jd_sample_yota_2', 'img3', 'jd_sample_yota_3', 'img2']

WIDTH = 180
HEIGHT = 90
FPS = 30

clips_and_imgs = []
for name in clips_order:
    print(name)
    start_sec, end_sec = concat_img_dict[name]
    if name.startswith('img'):
        print('\t\timg')
        duration = end_sec - start_sec
        ifile_img_path = f'{input_img_dir}/{name}.jpeg'
        clip = mp.ImageClip(ifile_img_path, duration=duration)
        clip.set_fps(FPS)
    else:
        print('\t\tvideo')
        ifile_video_path = f'{input_video_dir}/{name}.mp4'
        clip = mp.VideoFileClip(ifile_video_path).subclip(start_sec, end_sec)
        print('\t\tclip fps', clip.fps)
    clips_and_imgs.append(clip.resize(width=WIDTH, height=HEIGHT))

# save result video
final_clip = mp.concatenate_videoclips(clips_and_imgs, method='compose')
final_clip.write_videofile(ofile_video_img_path,
                           codec='libx264', 
                           audio_codec='aac', 
                           temp_audiofile='temp-audio.m4a', 
                           remove_temp=True
                          )

img1
		img
jd_sample_yota_2
		video
		clip fps 30.0
img3
		img
jd_sample_yota_3
		video


chunk:  18%|█▊        | 116/662 [00:00<00:00, 1158.62it/s, now=None]

		clip fps 30.0
img2
		img
Moviepy - Building video ../data/output/jd_videos_all_img.mp4.
MoviePy - Writing audio in temp-audio.m4a


t:   9%|▉         | 92/1050 [00:00<00:01, 901.23it/s, now=None]     

MoviePy - Done.
Moviepy - Writing video ../data/output/jd_videos_all_img.mp4



Moviepy - Done !
Moviepy - video ready ../data/output/jd_videos_all_img.mp4
CPU times: user 17 s, sys: 2.01 s, total: 19 s
Wall time: 28.5 s


In [15]:
final_clip.ipython_display(logger='none')

chunk:   0%|          | 0/662 [00:00<?, ?it/s, now=None]

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


t:   8%|▊         | 82/1050 [00:00<00:01, 818.72it/s, now=None]    

MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
